In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

Tesla T4


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import tqdm

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=40)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=40)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', weights='ResNet152_Weights.DEFAULT')
model = model.to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.975)

total_loss = 0
total_size = 0
model.train()
criterion=nn.CrossEntropyLoss()
best_loss = 100
for epoch in range(100):

  for batch_idx, (data, target) in tqdm.tqdm(enumerate(trainloader)):
      data, target = data.to('cuda:0'), target.to('cuda:0')
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      total_loss += loss.item()
      total_size += data.size(0)
      loss.backward()
      optimizer.step()
      if batch_idx % 100 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(trainloader.dataset),
              100. * batch_idx / len(trainloader), total_loss / total_size))

  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in testloader:
      data, target = data.to('cuda:0'), target.to('cuda:0')
      output = model(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1,keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

  test_loss /= len(testloader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(testloader.dataset),
      100. * correct / len(testloader.dataset)
  ))
  if test_loss < best_loss:
    print("new loss, save!")
    torch.save(model, 'resnet.model')
    best_loss = test_loss
  scheduler.step(epoch)




In [3]:
def eval_model(model, dataset_loader, img_func=None):
    tp, tp_5, counter = 0., 0., 0.
    for imgs, labels in tqdm.tqdm(dataset_loader, desc="Validating..."):
        imgs = imgs.to('cuda:0')
        labels = labels.to('cuda:0')
        if img_func is not None:
            imgs = imgs + img_func(model, imgs, labels)
        with torch.no_grad():
            preds = model(imgs)
        tp += (preds.argmax(dim=-1) == labels).sum()
        tp_5 += (preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        counter += preds.shape[0]
    acc = tp.float().item()/counter
    top5 = tp_5.float().item()/counter
    print(f"Top-1 error: {(100.0 * (1 - acc)):4.2f}%")
    print(f"Top-5 error: {(100.0 * (1 - top5)):4.2f}%")
    return acc, top5

In [4]:
# First, upload the file or copy the file into your google drive.
# And mount your drive.
import torch
import torch.nn as nn

# Then load the model
model = torch.load('/content/drive/MyDrive/CS255-Lab4/resnet.model', map_location=torch.device('cpu')) # e.g. /content/drive/MyDrive/CS255/resnet.model
#print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device) # if GPU available

In [5]:
import pickle

with open('/content/drive/MyDrive/CS255-Lab4/test_set.pkl', 'rb') as f:
  testset = pickle.load(f)
batch_size = 4
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

In [7]:
def fast_gradient_sign_method(model, input_imgs, labels, epsilon=0.08):
  # Initialize a zero tensor for perturbations with the same size as input_imgs
  perturbation = torch.zeros_like(input_imgs)
  perturbation.requires_grad = True

  # Forward pass: compute the model's prediction error using Cross-Entropy Loss
  prediction = model(input_imgs + perturbation)
  loss = nn.CrossEntropyLoss()(prediction, labels)

  # Backward pass: calculate gradients of the loss w.r.t perturbation
  loss.backward()

  # Calculate the sign of the gradients and scale by epsilon
  perturbed_sign = epsilon * perturbation.grad.data.sign()

  return perturbed_sign

  # This is NOT a runnable code!
  # preds = model(input_imgs.to(device)) # first get the prediction
  # loss = CrossEntropy_loss(input_imgs, labels) # second get the loss
  # perturbations = torch.sign(gradiant(input_imgs, loss)) # then get the perturbation
  # adversarial_samples = input_imgs + epsilon * perturbations # update the image
  # return adversarial_samples

In [9]:
import tqdm
eval_model(model, testloader, fast_gradient_sign_method)

Validating...: 100%|██████████| 25/25 [00:04<00:00,  6.00it/s]

Top-1 error: 63.00%
Top-5 error: 27.00%


(0.37, 0.73)

In [13]:
def pgd(model, input_imgs, labels, epsilon=0.06, alpha=1e4, num_iter=100):
    # Initialize perturbation as a zero tensor with gradients enabled
    perturbation = torch.zeros_like(input_imgs, requires_grad=True)

    # Iteratively apply gradient ascent to the perturbation
    for iteration in range(num_iter):
        # Compute the loss of the model's prediction with perturbed images
        predictions = model(input_imgs + perturbation)
        loss = nn.CrossEntropyLoss()(predictions, labels)

        # Perform backpropagation to compute gradients of the loss w.r.t. perturbation
        loss.backward()

        # Update the perturbation using the gradient, scaled by alpha
        # and ensure it's within the epsilon constraint
        perturbation.data = (perturbation + alpha * perturbation.grad.data / input_imgs.shape[0]).clamp(-epsilon, epsilon)

        # Reset gradients for the next iteration
        perturbation.grad.zero_()

    # Return the final perturbation detached from the current graph
    return perturbation.detach()

    # # This is NOT a runnable code!
    # delta = torch.zeros_like(X, requires_grad=True) # in the first iteration delta=0
    # for t in tqdm.tqdm(range(num_iter)):
    #     loss = CrossEntropyLoss()(model(input_imgs + delta), labels) # get the loss
    #     updated_delta = (delta + input_imgs*alpha*delta) # calculate the delta
    #     delta = updated_delta.clamp(-epsilon, +epsilon) # clamp the delta
    # adversarial_samples = delta # update the image
    # return adversarial_samples

In [14]:
import tqdm
eval_model(model, testloader, pgd)

Validating...: 100%|██████████| 25/25 [02:33<00:00,  6.16s/it]

Top-1 error: 64.00%
Top-5 error: 22.00%


(0.36, 0.78)

In [10]:
def save_imgs(model, dataset_loader, img_func=fast_gradient_sign_method):
    X_train = []
    Y_train = []
    for imgs, labels in tqdm.tqdm(dataset_loader, desc="saving..."):
        imgs = imgs.to('cuda:0')
        labels_gpu = labels.to('cuda:0')
        imgs = imgs + img_func(model, imgs, labels_gpu)
        for i in range(dataset_loader.batch_size):
            X_train.append(imgs[i].detach().cpu())
            Y_train.append(labels[i].detach())
    sampled_test_data = [(X,Y) for X,Y in zip(X_train, Y_train)]
    if img_func == fast_gradient_sign_method:
      with open("fgsm.pkl", "wb") as f:
        pickle.dump(sampled_test_data, f)
    elif img_func == pgd:
      with open("pgd.pkl", "wb") as f:
        pickle.dump(sampled_test_data, f)
    else:
      print("You are using incorrect function")

In [11]:
def self_check(model, result_set='fgsm.pkl'):
    with open(result_set, "rb") as f:
        resultset = pickle.load(f)
    with open("/content/drive/MyDrive/CS255-Lab4/test_set.pkl", "rb") as f2:
        testset = pickle.load(f2)
    eval_set = []
    for org, adv in zip(testset,resultset):
        assert org[1] == adv[1]
        eval_set.append((org[0], adv[0], org[1]))
    testloader = torch.utils.data.DataLoader(eval_set, batch_size=4,
                                            shuffle=False, num_workers=40) # colab uses num_workers=2


    org_tp, org_tp_5, adv_tp, adv_tp_5, counter, diff = 0., 0., 0., 0., 0., 0.
    for imgs, adv_imgs, labels in tqdm.tqdm(testloader, desc="Validating..."):
        imgs = imgs.to('cuda:0')
        adv_imgs = adv_imgs.to('cuda:0')
        labels = labels.to('cuda:0')
        with torch.no_grad():
            org_preds = model(imgs)
            adv_preds = model(adv_imgs)
        diff += torch.sum(torch.abs(adv_imgs-imgs))/(3*32*32) #[4, 3, 32, 32]
        org_tp += (org_preds.argmax(dim=-1) == labels).sum()
        org_tp_5 += (org_preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        adv_tp += (adv_preds.argmax(dim=-1) == labels).sum()
        adv_tp_5 += (adv_preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        counter += org_preds.shape[0]
    org_acc = org_tp.float().item()/counter
    org_top5 = org_tp_5.float().item()/counter
    adv_acc = adv_tp.float().item()/counter
    adv_top5 = adv_tp_5.float().item()/counter


    result = "correct" if org_acc-adv_acc >= 0.2 and org_top5-adv_top5 >= 0.15 and diff/counter <= 0.1 else "need to improve"
    print(f"Top-1 error on original samples: {(100.0 * (1 - org_acc)):4.2f}%; on adverserial samples: {(100.0 * (1 - adv_acc)):4.2f}%")
    print(f"Top-5 error on original samples: {(100.0 * (1 - org_top5)):4.2f}% on adverserial samples: {(100.0 * (1 - adv_top5)):4.2f}%")
    print(result)
    if (diff/counter > 0.1):
        print("epsilon > 0.1")


In [12]:
save_imgs(model, testloader, fast_gradient_sign_method)
self_check(model, 'fgsm.pkl')

saving...: 100%|██████████| 25/25 [00:01<00:00, 15.45it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validating...: 100%|██████████| 25/25 [00:03<00:00,  7.81it/s]

Top-1 error on original samples: 40.00%; on adverserial samples: 63.00%
Top-5 error on original samples: 9.00% on adverserial samples: 27.00%
correct


In [15]:
save_imgs(model, testloader, pgd)
self_check(model, 'pgd.pkl')

Validating...: 100%|██████████| 25/25 [00:02<00:00,  8.91it/s]

Top-1 error on original samples: 40.00%; on adverserial samples: 69.00%
Top-5 error on original samples: 9.00% on adverserial samples: 26.00%
correct
